# MQTT client test - Paho
https://pypi.python.org/pypi/paho-mqtt/1.2  
https://mosquitto.org/   
http://www.hivemq.com/blog/mqtt-essentials/  

Start a Mosquitto container first. For example:
- Use `codes\_demo\1_start_broker.sh` to start a Mosquitto container on Raspberry Pi.
- Config files are in `mqtt_config\mqtt`.
- set `allow_anonymous true` in `mqtt_config\mqtt\config\mosquitto.conf` to allow anonymous client.

In [ ]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, '..\\codes\\shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, '..\\codes\\node')))
import config
import worker_config

In [ ]:
host = config.BROKER_HOST
topic_root = "test"
client_id = worker_config.WORKER_NAME
auth = {'username': config.USERNAME,
        'password': config.PASSWORD}
qos_level = config.QOS_LEVEL

In [ ]:
topic = topic_root + '/test'
message = 'Hello'
publish.single(topic, message, qos = qos_level, 
               hostname = host, auth = auth,
               client_id = client_id)

## Getting Started
Here is a very simple example that subscribes to the broker $SYS topic tree and prints out the resulting messages:

In [ ]:
import paho.mqtt.client as mqtt

client = mqtt.Client(client_id = client_id)  # , clean_session=True, userdata=None, protocol=MQTTv311, transport="tcp")
client.max_inflight_messages_set(1)
# client.will_set(topic, payload=None, qos=qos_level, retain=False)


# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code {}".format(str(rc)))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$SYS/broker/messages/#", qos = qos_level)

def on_disconnect(client, userdata, rc):
    print("Disconnected with result code {}".format(str(rc)))

def on_publish(client, userdata, mid):
    print("On publish {}".format(mid))

def on_subscribe(client, userdata, mid, granted_qos):
    print("On subscribe, mid:{}, granted_qos:{}".format(mid, granted_qos))
        
def on_unsubscribe(client, userdata, mid):
    print("On unsubscribe {}".format(mid))
        
# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, message):
    print('Message topic: {}, payload: {}'.format(message.topic, str(message.payload)))
    print('Message:\n{}\n'.format(message.__dict__))

def on_log(client, userdata, level, buf):
    print("On log level: {}, buf: {}".format(level, buf))
    
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_publish = on_publish
client.on_subscribe = on_subscribe
client.on_unsubscribe = on_unsubscribe
client.on_message = on_message


client.username_pw_set(**auth)

In [ ]:
client.connect(host = host)  # , port=1883, keepalive=60, bind_address="")
# client.reconnect()

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

In [ ]:
client.disconnect()
client = None

## Client
You can use the client class as an instance, within a class or by subclassing. The general usage flow is as follows:

- Create a client instance
- Connect to a broker using one of the connect*() functions
- Call one of the loop*() functions to maintain network traffic flow with the broker
- Use subscribe() to subscribe to a topic and receive messages
- Use publish() to publish messages to the broker
- Use disconnect() to disconnect from the broker  

Callbacks will be called to allow the application to process events as necessary. These callbacks are described below

### Constructor / reinitialise
Client()
```
Client(client_id="", clean_session=True, userdata=None, protocol=MQTTv311, transport="tcp")
```

### Option functions
These functions represent options that can be set on the client to modify its behaviour. In the majority of cases this must be done before connecting to a broker.  

max_inflight_messages_set()

```
max_inflight_messages_set(self, inflight)
```

### Connect / reconnect / disconnect
connect()
```
connect(host, port=1883, keepalive=60, bind_address="")
```

### Network loop
These functions are the driving force behind the client. If they are not called, incoming network data will not be processed and outgoing network data may not be sent in a timely fashion. There are four options for managing the network loop. Three are described here, the fourth in “External event loop support” below. Do not mix the different loop functions.

loop()

```
loop(timeout=1.0, max_packets=1)
```

loop_forever()
```
loop_forever(timeout=1.0, max_packets=1, retry_first_connection=False)
```

### Publishing
Send a message from the client to the broker.

publish()  
```
publish(topic, payload=None, qos=0, retain=False)
```

### Subscribe / Unsubscribe
subscribe()
```
subscribe(topic, qos=0)
```

### Callbacks
on_connect()
```
on_connect(client, userdata, flags, rc)
```

### message_callback_add()

This function allows you to define callbacks that handle incoming messages for specific subscription filters, including with wildcards. This lets you, for example, subscribe to sensors/# and have one callback to handle sensors/temperature and another to handle sensors/humidity.
```
message_callback_add(sub, callback)
```
sub  
`the subscription filter to match against for this callback. Only one callback may be defined per literal sub string  `  
callback  
`the callback to be used. Takes the same form as the on_message callback.`  

**If using message_callback_add() and on_message, only messages that do not match a subscription specific filter will be passed to the on_message callback.**

message_callback_remove()

Remove a topic/subscription specific callback previously registered using message_callback_add().
```
message_callback_remove(sub)
```
sub
the subscription filter to remove

## External event loop support
loop_read() 
```
loop_read(max_packets=1)
```
 
Call when the socket is ready for reading. max_packets is obsolete and should be left unset.

loop_write()
```
loop_write(max_packets=1)
```
Call when the socket is ready for writing. max_packets is obsolete and should be left unset.

loop_misc()
```
loop_misc()
```
Call every few seconds to handle message retrying and pings.

socket()
```
socket()
```
Returns the socket object in use in the client to allow interfacing with other event loops.

want_write()
```
want_write()
```
Returns true if there is data waiting to be written, to allow interfacing the client with other event loops.

## Global helper functions
The client module also offers some global helper functions.

topic_matches_sub(sub, topic) can be used to check whether a topic matches a subscription.

For example:

- the topic foo/bar would match the subscription foo/# or +/bar

- the topic non/matching would not match the subscription non/+/+

connack_string(connack_code) returns the error string associated with a CONNACK result.

error_string(mqtt_errno) returns the error string associated with a Paho MQTT error number.

## Publish
This module provides some helper functions to allow straightforward publishing of messages in a one-shot manner. In other words, they are useful for the situation where you have a single/multiple messages you want to publish to a broker, then disconnect with nothing else required.

The two functions provided are single() and multiple().

### Single
Publish a single message to a broker, then disconnect cleanly.
```
single(topic, payload=None, qos=0, retain=False, hostname="localhost",
    port=1883, client_id="", keepalive=60, will=None, auth=None, tls=None,
    protocol=mqtt.MQTTv311, transport="tcp")
    ```
Publish Single Function arguments

topic  
the only required argument must be the topic string to which the payload will be published.
payload  
the payload to be published. If “” or None, a zero length payload will be published.
qos  
the qos to use when publishing, default to 0.
retain  
set the message to be retained (True) or not (False).
hostname  
a string containing the address of the broker to connect to. Defaults to localhost.
port  
the port to connect to the broker on. Defaults to 1883.
client_id  
the MQTT client id to use. If “” or None, the Paho library will generate a client id automatically.
keepalive  
the keepalive timeout value for the client. Defaults to 60 seconds.
will  
a dict containing will parameters for the client:

will = {‘topic’: “<topic>”, ‘payload’:”<payload”>, ‘qos’:<qos>, ‘retain’:<retain>}.

Topic is required, all other parameters are optional and will default to None, 0 and False respectively.

Defaults to None, which indicates no will should be used.

auth  
a dict containing authentication parameters for the client:

auth = {‘username’:”<username>”, ‘password’:”<password>”}

Username is required, password is optional and will default to None if not provided.

Defaults to None, which indicates no authentication is to be used.

tls  
a dict containing TLS configuration parameters for the client:

dict = {‘ca_certs’:”<ca_certs>”, ‘certfile’:”<certfile>”, ‘keyfile’:”<keyfile>”, ‘tls_version’:”<tls_version>”, ‘ciphers’:”<ciphers”>}

ca_certs is required, all other parameters are optional and will default to None if not provided, which results in the client using the default behaviour - see the paho.mqtt.client documentation.

Defaults to None, which indicates that TLS should not be used.

protocol  
choose the version of the MQTT protocol to use. Use either MQTTv31 or MQTTv311.
transport  
set to “websockets” to send MQTT over WebSockets. Leave at the default of “tcp” to use raw TCP.
Publish Single Example  
```
import paho.mqtt.publish as publish
publish.single("paho/test/single", "payload", hostname="iot.eclipse.org")
```

### Multiple
Publish multiple messages to a broker, then disconnect cleanly.
```
multiple(msgs, hostname="localhost", port=1883, client_id="", keepalive=60,
    will=None, auth=None, tls=None, protocol=mqtt.MQTTv311, transport="tcp")
    ```
Publish Multiple Function arguments

msgs  
a list of messages to publish. Each message is either a dict or a tuple.

If a dict, only the topic must be present. Default values will be used for any missing arguments. The dict must be of the form:

**msg = {‘topic’:”<topic>”, ‘payload’:”<payload>”, ‘qos’:<qos>, ‘retain’:<retain>}**

topic must be present and may not be empty. If payload is “”, None or not present then a zero length payload will be published. If qos is not present, the default of 0 is used. If retain is not present, the default of False is used.

If a tuple, then it must be of the form:

(“<topic>”, “<payload>”, qos, retain)

See single() for the description of hostname, port, client_id, keepalive, will, auth, tls, protocol, transport.

Publish Multiple Example
```
import paho.mqtt.publish as publish

msgs = [{'topic':"paho/test/multiple", 'payload':"multiple 1"},
    ("paho/test/multiple", "multiple 2", 0, False)]
publish.multiple(msgs, hostname="iot.eclipse.org")
```

## Subscribe
This module provides some helper functions to allow straightforward subscribing and processing of messages.

The two functions provided are simple() and callback().

### Simple
Subscribe to a set of topics and return the messages received. This is a **blocking** function.
```
simple(topics, qos=0, msg_count=1, retained=False, hostname="localhost",
    port=1883, client_id="", keepalive=60, will=None, auth=None, tls=None,
    protocol=mqtt.MQTTv311)
    ```
    
Simple Subscribe Function arguments

topics  
the only required argument is the topic string to which the client will subscribe. This can either be a string or a list of strings if multiple topics should be subscribed to.
qos  
the qos to use when subscribing, defaults to 0.
**msg_count  
the number of messages to retrieve from the broker. Defaults to 1. If 1, a single MQTTMessage object will be returned. If >1, a list of MQTTMessages will be returned.**
retained  
set to True to consider retained messages, set to False to ignore messages with the retained flag set.
hostname  
a string containing the address of the broker to connect to. Defaults to localhost.
port  
the port to connect to the broker on. Defaults to 1883.
client_id  
the MQTT client id to use. If “” or None, the Paho library will generate a client id automatically.
keepalive  
the keepalive timeout value for the client. Defaults to 60 seconds.
will  
a dict containing will parameters for the client:

will = {‘topic’: “<topic>”, ‘payload’:”<payload”>, ‘qos’:<qos>, ‘retain’:<retain>}.

Topic is required, all other parameters are optional and will default to None, 0 and False respectively.

Defaults to None, which indicates no will should be used.

auth  
a dict containing authentication parameters for the client:

auth = {‘username’:”<username>”, ‘password’:”<password>”}

Username is required, password is optional and will default to None if not provided.

Defaults to None, which indicates no authentication is to be used.

tls  
a dict containing TLS configuration parameters for the client:

dict = {‘ca_certs’:”<ca_certs>”, ‘certfile’:”<certfile>”, ‘keyfile’:”<keyfile>”, ‘tls_version’:”<tls_version>”, ‘ciphers’:”<ciphers”>}

ca_certs is required, all other parameters are optional and will default to None if not provided, which results in the client using the default behaviour - see the paho.mqtt.client documentation.

Defaults to None, which indicates that TLS should not be used.

protocol  
choose the version of the MQTT protocol to use. Use either MQTTv31 or MQTTv311.
Simple Example
```
import paho.mqtt.subscribe as subscribe

msg = subscribe.simple("paho/test/simple", hostname="iot.eclipse.org")
print("%s %s" % (msg.topic, msg.payload)) 
```

### Using Callback
Subscribe to a set of topics and process the messages received using a user provided callback.
```
callback(callback, topics, qos=0, userdata=None, hostname="localhost",
    port=1883, client_id="", keepalive=60, will=None, auth=None, tls=None,
    protocol=mqtt.MQTTv311)
Callback Subscribe Function arguments
```
callback  
an “on_message” callback that will be used for each message received, and of the form  

def on_message(client, userdata, message)  
topics  
the topic string to which the client will subscribe. This can either be a string or a list of strings if multiple topics should be subscribed to.  
qos  
the qos to use when subscribing, defaults to 0.  
userdata  
a user provided object that will be passed to the on_message callback when a message is received.  
See simple() for the description of hostname, port, client_id, keepalive, will, auth, tls, protocol.  

Callback Example  
```
import paho.mqtt.subscribe as subscribe

def on_message_print(client, userdata, message):
    print("%s %s" % (message.topic, message.payload))

subscribe.callback(on_message_print, "paho/test/callback", hostname="iot.eclipse.org")
```